In [4]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error

Generating the dataset and splitting the data

In [5]:
# --- defining the Franke function
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

# --- Generating dataset
n = 100 # the square root of the number of datapoints
x, y = np.linspace(0,1,n), np.linspace(0,1,n)
x, y = np.meshgrid(x, y)
x = x.ravel()
y = y.ravel()

k = .01 # noise coefficient
z = FrankeFunction(x,y) + k*np.random.randn(n**2) # target variable with standard normal noise

X = np.column_stack((x,y))

X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=0)

### Scoring for Optimal Model Parameters from Gridsearch of Own Implementation

In [8]:
# Defining the neural network
class RegressionNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressionNet, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)        # one hidden layer
        self.output = nn.Linear(hidden_size, output_size)

    # defining the forward pass
    def forward(self, x):
        x = torch.sigmoid(self.hidden(x))       # sigmoid activation
        x = self.output(x)                      # 'identity' activation
        return x

# Defining the sizes of the layers
input_size = X_train.shape[1] 
hidden_size = 50  
output_size = 1  

# Initialize the model, loss function, and optimizer
model = RegressionNet(input_size, hidden_size, output_size)
criterion = nn.MSELoss()                                        # using MSE as a cost function
optimizer = optim.SGD(model.parameters(), lr=0.1)               # Using SGD with a learning rate of 0.1

# Standardizing the data and convert it to Pytorch tensors
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
z_train_tensor = torch.tensor(z_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
z_test_tensor = torch.tensor(z_test, dtype=torch.float32).view(-1, 1)

# Creating data loaders
train_dataset = TensorDataset(X_train_tensor, z_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)           # batch size 32

# Training loop
epochs = 500
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    
    for X_batch, z_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, z_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    
    train_loss /= len(train_loader.dataset)
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}')


# Evaluating the model on the test data
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).squeeze()
    test_loss = mean_squared_error(z_test, predictions)
print(f'Test MSE: {test_loss:.4f}')

Epoch 100/500, Loss: 0.0057
Epoch 200/500, Loss: 0.0024
Epoch 300/500, Loss: 0.0018
Epoch 400/500, Loss: 0.0015
Epoch 500/500, Loss: 0.0013
Test MSE: 0.0012


### Comparison with own implementation

Calculating the MSE as a function of the learning rate and saving the results

In [9]:
# Defining the neural network
class RegressionNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressionNet, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)        # one hidden layer
        self.output = nn.Linear(hidden_size, output_size)

    # defining the forward pass
    def forward(self, x):
        x = torch.sigmoid(self.hidden(x))       # sigmoid activation
        x = self.output(x)                      # 'identity' activation
        return x

# Defining the sizes of the layers
input_size = X_train.shape[1] 
hidden_size = 50  
output_size = 1  

# Initialize the model, loss function, and optimizer
model = RegressionNet(input_size, hidden_size, output_size)
criterion = nn.MSELoss()                                        # using MSE as a cost function
optimizer = optim.SGD(model.parameters(), lr=0.1)               # Using SGD with a learning rate of 0.1

# Standardizing the data and convert it to Pytorch tensors
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
z_train_tensor = torch.tensor(z_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
z_test_tensor = torch.tensor(z_test, dtype=torch.float32).view(-1, 1)

# Creating data loaders
train_dataset = TensorDataset(X_train_tensor, z_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)           # batch size 32

# Training loop
epochs = 10
N = 200
learning_rates = np.logspace(-4, -1, N)
scores = np.zeros(N)
for idx, lr in enumerate(learning_rates):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        
        for X_batch, z_batch in train_loader:
            optimizer = optim.SGD(model.parameters(), lr=lr)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, z_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        if (epoch + 1) % 50 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}')

    # Evaluate the model on test data
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor).squeeze()
        test_loss = mean_squared_error(z_test, predictions)

    scores[idx] = test_loss

In [10]:
np.save('Results/regression_pytorch_learning_rate.npy', scores)